# **1. Demo Básica de Retrieval-Augmented Generation (BasicRAG) con Gemini:**

## ¿Cómo funciona?

En este ejemplo veremos cómo funciona un sistema básico de RAG (Retrieval-Augmented Generation).
La idea principal es:

1. Subir un documento de texto (ejemplo: un archivo con información de física cuántica o historia).

2. Dividir el documento en fragmentos pequeños (chunks).

3. Convertir esos fragmentos en embeddings (vectores numéricos que representan el significado del texto).

4. Guardar los embeddings en una base de datos vectorial (ChromaDB).

5. Hacer una pregunta en lenguaje natural → el sistema busca los fragmentos más relevantes.

6. Gemini responde usando esos fragmentos como contexto, para dar una respuesta más precisa y confiable.

Esto es lo que hace especial a RAG: el modelo no depende solo de su memoria entrenada, sino que consulta información externa que nosotros le damos.

## **Preparación del entorno:**

**Antes de ejecutar el código:**

1. **Obtener tu API Key de Gemini:**

- Entra a 👉 Google AI Studio: https://aistudio.google.com/.

- Crea una clave desde la sección API Keys.

- Copia tu API Key y reemplaza en la línea: **os.environ["GEMINI_API_KEY"] = "TU_API_KEY_AQUI"**


**Este paso solo es necesario si la que está puesta falla, o si cada estudiante quiere usar su propia clave.**

2. **Archivos de prueba disponibles en Drive (Debajo se indican los documentos utilizados en la demo en calse): https://drive.google.com/drive/folders/1uF7-oSMpzSdID2ltQf9NsU5Snf16fHfx?usp=sharing**

- cuantica.txt → introducción a la física cuántica. **(Usado en al demo en clase)**.

- innovadores.txt → introducción a la física cuántica. **(Usado en al demo en clase)**.

- historia_Colombia.txt → texto sobre historia de Colombia.

- historia_internet.txt → documento con explicación básica de Einstein.

👉 **Suban uno de estos archivos (o el suyo propio) cuando el código lo pida.**

In [32]:
!pip install evaluate==0.4.5
!pip install rouge_score

In [ ]:
# ================= DEMO RAG CON GEMINI ==================
!pip install chromadb sentence-transformers google-generativeai

import os
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import chromadb
from google.colab import files

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 111.6 MB/s eta 

In [46]:
# ========= 1. CONFIGURACIÓN DE GEMINI =========
# 👉 Paso previo: obtener API Key en https://aistudio.google.com/
#  Cambia tu API Key SOLO si la actual no funciona.
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('gemini')
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

In [34]:
# ========= 2. SUBIR ARCHIVO =========
print("Sube un archivo de texto con información (ej: cuantica.txt)")
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
with open(file_name, "r", encoding="utf-8") as f:
    text = f.read()


Sube un archivo de texto con información (ej: cuantica.txt)


KeyboardInterrupt: 

In [35]:
# ========= 3. FUNCIÓN DE CHUNKING =========
def chunk_text(text, chunk_size=80, overlap=20):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

docs = chunk_text(text, chunk_size=80, overlap=20)

print("Ejemplo de 3 chunks creados:")
for c in docs[:3]:
    print("-", c, "\n")

Ejemplo de 3 chunks creados:
- Elon Musk fundó Tesla, una compañía dedicada a la fabricación de automóviles eléctricos y soluciones de energía renovable. También fundó SpaceX, una empresa de exploración espacial que desarrolla cohetes y satélites. Además, participó en la creación de PayPal, un sistema de pagos en línea que revolucionó las transacciones digitales. Steve Jobs fue cofundador de Apple, empresa reconocida por sus productos tecnológicos innovadores como el iPhone, iPad y MacBook. Apple ha sido clave en el desarrollo de la industria de los 

- tecnológicos innovadores como el iPhone, iPad y MacBook. Apple ha sido clave en el desarrollo de la industria de los dispositivos inteligentes. Jeff Bezos fundó Amazon, inicialmente como una librería en línea, que evolucionó hacia una de las mayores plataformas de comercio electrónico y servicios en la nube a través de Amazon Web Services (AWS). Mark Zuckerberg creó Facebook, una red social que transformó la comunicación digital, la pub

In [36]:
# ========= 4. CREAR EMBEDDINGS Y BASE VECTORIAL =========
embedder = SentenceTransformer("all-MiniLM-L6-v2")
client = chromadb.Client()
collection = client.get_or_create_collection("physics_chunks")

embeddings = embedder.encode(docs).tolist()
for i, d in enumerate(docs):
    collection.add(documents=[d], embeddings=[embeddings[i]], ids=[str(i)])



In [37]:
# ========= 5. CONSULTA Y RETRIEVAL =========
# 👉 Aquí puedes cambiar la pregunta y experimentar ========================================
# Ejemplos para probar:
# query = "¿Qué explica la teoría de la relatividad?"
# query = "¿Cuál fue un hecho clave en la independencia de Colombia?"
query = "¿Qué científico propuso un modelo atómico en 1913?"
q_embed = embedder.encode([query]).tolist()
results = collection.query(query_embeddings=q_embed, n_results=3)
retrieved_context = " ".join(results['documents'][0])

print("\n🔹 Chunks relevantes recuperados:")
for doc in results['documents'][0]:
    print("-", doc)

print("\n🔹 Contexto recuperado:")
print(retrieved_context)


🔹 Chunks relevantes recuperados:
- Max Planck introdujo en 1900 la idea de que la energía no se emite de manera continua, sino en cuantos discretos. Este fue el inicio de la teoría cuántica. Albert Einstein en 1905 explicó el efecto fotoeléctrico utilizando el concepto de cuantos de luz, lo que posteriormente llamó fotones. Niels Bohr en 1913 propuso su modelo atómico, donde los electrones orbitaban en niveles de energía cuantizados alrededor del núcleo. Werner Heisenberg enunció en 1927 el principio de incertidumbre, que indica
- orbitaban en niveles de energía cuantizados alrededor del núcleo. Werner Heisenberg enunció en 1927 el principio de incertidumbre, que indica que no es posible conocer con precisión la posición y el momento de una partícula al mismo tiempo. Erwin Schrödinger desarrolló la ecuación de onda en 1926, fundamental para describir el comportamiento cuántico de las partículas. Richard Feynman contribuyó a la electrodinámica cuántica y popularizó el uso de diagramas 

In [47]:
# ========= 6. GEMINI PARA RESPONDER CON CONTEXTO =========
model = genai.GenerativeModel("models/gemini-1.5-flash-latest")

response = model.generate_content(
    f"UBasándote en el siguiente contexto, responde la pregunta y añade una breve explicación adicional desde tu conocimiento si es relevante:.\n\nContexto:\n{retrieved_context}\n\nPregunta: {query}"
)

response_no_context = model.generate_content(
    f"UBasándote en el siguiente contexto, responde la pregunta y añade una breve explicación adicional desde tu conocimiento si es relevante:.\n\nPregunta: {query}"
)

response_chat = "El científico que propuso un modelo atómico en **1913 fue Niels Bohr**. Bohr planteó que los electrones orbitan alrededor del núcleo en niveles de energía cuantizados, lo que permitió explicar fenómenos como las líneas espectrales del hidrógeno. Este modelo fue un gran avance respecto al de Rutherford, ya que incorporó ideas de la teoría cuántica para describir la estabilidad del átomo. De manera adicional, aunque luego fue superado por la mecánica cuántica de Schrödinger y Heisenberg, el modelo de Bohr sigue siendo muy útil en la enseñanza porque introduce de forma sencilla el concepto de niveles de energía y la transición de electrones con emisión o absorción de fotones."

print("\n🔹 Respuesta generada por Gemini:")
print(response.text)


print("\n🔹 Respuesta generada por Gemini sin contexto:")
print(response_no_context.text)


print("\n🔹 Respuesta generada por Chatgpt:")
print(response_chat)


🔹 Respuesta generada por Gemini:
Niels Bohr propuso un modelo atómico en 1913.  Este modelo incorporaba la idea de cuantización de la energía, postulada previamente por Planck, para explicar la estabilidad del átomo y las líneas espectrales del hidrógeno.  A diferencia de los modelos atómicos anteriores, el de Bohr planteaba que los electrones orbitaban el núcleo en niveles de energía discretos y definidos, y que la emisión o absorción de energía ocurría cuando un electrón saltaba entre estos niveles.  Si bien este modelo tenía limitaciones y fue posteriormente superado por la mecánica cuántica, fue un paso crucial en la comprensión de la estructura atómica.


🔹 Respuesta generada por Gemini sin contexto:
Niels Bohr propuso un modelo atómico en 1913.

**Explicación adicional:** El modelo atómico de Bohr mejoró significativamente el modelo de Rutherford al incorporar la teoría cuántica de Planck.  A diferencia del modelo de Rutherford, que dejaba sin explicar la estabilidad de los átom

### Métricas de evaluación

In [51]:
from evaluate import load


def evaluate_response(response, reference_response):
  # Generated response from Gemini with context
  generated_response_context = response.text
  # Reference response from chatgpt
  reference_response = response_chat

  # Calculate BLEU score for context
  bleu = load("bleu")
  bleu_score_context = bleu.compute(predictions=[generated_response_context], references=[reference_response])
  print(f"\n🔹 BLEU score (Gemini with context): {bleu_score_context['bleu']:.4f}")
  print("🔹 BLEU precision scores (Gemini with context):")
  for i, precision in enumerate(bleu_score_context['precisions']):
      print(f"- Precision@{i+1}: {precision:.4f}")

  # Calculate ROUGE score for context
  rouge = load("rouge")
  rouge_score_context = rouge.compute(predictions=[generated_response_context], references=[reference_response])
  print("\n🔹 ROUGE score (Gemini with context):")
  for key, value in rouge_score_context.items():
      print(f"- {key}: {value:.4f}")

  return bleu_score_context, rouge_score_context

In [52]:

def evaluate_response_no_context(response, reference_response):
  # Generated response from Gemini with no context
  generated_response_no_context = response_no_context.text
  # Reference response from chatgpt
  reference_response = response_chat

  # Calculate BLEU score for no context
  bleu = load("bleu")
  bleu_score_no_context = bleu.compute(predictions=[generated_response_no_context], references=[reference_response])
  print(f"\n🔹 BLEU score (Gemini without context): {bleu_score_no_context['bleu']:.4f}")

  # Calculate ROUGE score for no context
  rouge = load("rouge")
  rouge_score_no_context = rouge.compute(predictions=[generated_response_no_context], references=[reference_response])
  print("\n🔹 ROUGE score (Gemini without context):")
  for key, value in rouge_score_no_context.items():
      print(f"- {key}: {value:.4f}")

  return bleu_score_no_context, rouge_score_no_context

In [53]:
# ========= 8. COMPARACIÓN E INTERPRETACIÓN DE RESULTADOS =========

print("\n=== Comparación e Interpretación de las Métricas ===")

print("\nComparando las respuestas generadas con la referencia (ChatGPT):")

bue_score_context, rouge_score_context = evaluate_response(response, response_chat)
bue_score_no_context, rouge_score_no_context = evaluate_response_no_context(response_no_context, response_chat)

# Compare BLEU scores
print("\nResultados BLEU:")
print(f"- Gemini con contexto (BLEU): {bleu_score_context['bleu']:.4f}")
print(f"- Gemini sin contexto (BLEU): {bleu_score_no_context['bleu']:.4f}")

if bleu_score_context['bleu'] > bleu_score_no_context['bleu']:
    print("  -> Según el BLEU score general, la respuesta de Gemini con contexto es más similar a la referencia.")
elif bleu_score_context['bleu'] < bleu_score_no_context['bleu']:
     print("  -> Según el BLEU score general, la respuesta de Gemini sin contexto es más similar a la referencia.")
else:
    print("  -> Los BLEU scores generales son similares.")

print("\nResultados ROUGE:")
print(f"- Gemini con contexto (ROUGE-1): {rouge_score_context['rouge1']:.4f}")
print(f"- Gemini sin contexto (ROUGE-1): {rouge_score_no_context['rouge1']:.4f}")
print(f"- Gemini con contexto (ROUGE-L): {rouge_score_context['rougeL']:.4f}")
print(f"- Gemini sin contexto (ROUGE-L): {rouge_score_no_context['rougeL']:.4f}")


if rouge_score_context['rouge1'] > rouge_score_no_context['rouge1']:
    print("  -> Según ROUGE-1 (superposición de palabras individuales), la respuesta de Gemini con contexto es más similar a la referencia.")
elif rouge_score_context['rouge1'] < rouge_score_no_context['rouge1']:
    print("  -> Según ROUGE-1, la respuesta de Gemini sin contexto es más similar a la referencia.")
else:
    print("  -> Los ROUGE-1 scores son similares.")

if rouge_score_context['rougeL'] > rouge_score_no_context['rougeL']:
    print("  -> Según ROUGE-L (subsecuencia más larga, sensible al orden), la respuesta de Gemini con contexto es más similar a la referencia.")
elif rouge_score_context['rougeL'] < rouge_score_no_context['rougeL']:
    print("  -> Según ROUGE-L, la respuesta de Gemini sin contexto es más similar a la referencia.")
else:
     print("  -> Los ROUGE-L scores son similares.")


=== Comparación e Interpretación de las Métricas ===

Comparando las respuestas generadas con la referencia (ChatGPT):

🔹 BLEU score (Gemini with context): 0.0000
🔹 BLEU precision scores (Gemini with context):
- Precision@1: 0.0737
- Precision@2: 0.0080
- Precision@3: 0.0000
- Precision@4: 0.0000

🔹 ROUGE score (Gemini with context):
- rouge1: 0.1957
- rouge2: 0.0164
- rougeL: 0.1196
- rougeLsum: 0.1359

🔹 BLEU score (Gemini without context): 0.1400

🔹 ROUGE score (Gemini without context):
- rouge1: 0.5387
- rouge2: 0.2751
- rougeL: 0.2731
- rougeLsum: 0.2731

Resultados BLEU:
- Gemini con contexto (BLEU): 0.0904
- Gemini sin contexto (BLEU): 0.0896
  -> Según el BLEU score general, la respuesta de Gemini con contexto es más similar a la referencia.

Resultados ROUGE:
- Gemini con contexto (ROUGE-1): 0.1957
- Gemini sin contexto (ROUGE-1): 0.5387
- Gemini con contexto (ROUGE-L): 0.1196
- Gemini sin contexto (ROUGE-L): 0.2731
  -> Según ROUGE-1, la respuesta de Gemini sin contexto es m

# 2. **GraphRAG con Gemini + Neo4j: Consultando Grafos de Conocimiento:**

En este ejemplo usamos Gemini para extraer triples semánticos del texto (por ejemplo: (Elon Musk, fundó, Tesla)).

Luego esos triples se guardan en Neo4j, una base de datos orientada a grafos.
Después, podemos hacer consultas usando Cypher, el lenguaje de Neo4j, y finalmente Gemini genera una respuesta en lenguaje natural usando la información consultada.

Esto permite transformar un texto plano en un grafo de conocimiento consultable, lo que es muy útil para preguntas complejas que requieren relaciones entre entidades.

## **Preparación del entorno:**

Antes de ejecutar el código:

1. **Cuenta en Neo4j AuraDB (gratis): https://neo4j.com/product/auradb/**

- Ir a Neo4j AuraDB Free.

- Crear una cuenta y una base de datos gratuita.

- Copiar los datos de conexión (URI, Usuario, Contraseña).

**⚠️ Este paso, al igual que la creación de la API Key de Gemini, solo será necesario si las credenciales ya incluidas en el código no funcionan.
De esta manera, cada estudiante tendrá la opción de usar sus propias credenciales y su propia base de datos personalizada en caso de que sea necesario.**

2. **Nuevamente tener presente los archivos disponibles en Drive:https://drive.google.com/drive/folders/1uF7-oSMpzSdID2ltQf9NsU5Snf16fHfx?usp=sharing**

- cuantica.txt → introducción a la física cuántica. **(Usado en al demo en clase)**.

- innovadores.txt → introducción a la física cuántica. **(Usado en al demo en clase)**.

- historia_Colombia.txt → texto sobre historia de Colombia.

- historia_internet.txt → documento con explicación básica de Einstein.

👉 **Suban uno de estos archivos (o el suyo propio) cuando el código lo pida.****

## **Snippets de Cypher para practicar**

Los estudiantes pueden reemplazar el query y la pregunta con estos ejemplos (COPIAR Y PEGAR EN EL BLOQUE DE CODIGO DEL GRAPHRAG):

### **Para el documento innovadores.txt disponible en google drive:** ¿Cuáles son las empresas fundadas por Elon Musk y por Steve Jobs, a qué se dedica cada una y por qué es importante lo que hacen?


In [43]:
cypher_query = """
MATCH (p:Entidad)-[r:RELACION]->(c:Entidad)
RETURN p.name AS persona, r.tipo AS relacion, c.name AS compania
"""

### **Para el documento hisotria_internet.txt disponible en google drive:** ¿Cuál fue la importancia de Tim Berners-Lee en la historia de Internet?

In [44]:
cypher_query = """
MATCH (a:Entidad)-[r:RELACION]->(b:Entidad)
WHERE a.name = "Tim Berners-Lee"
RETURN a.name AS a, r.tipo AS relacion, b.name AS b
"""

In [58]:
# ================== INSTALACIÓN ==================
!pip install neo4j google-generativeai

import os
from neo4j import GraphDatabase
import google.generativeai as genai
from google.colab import files

# ================== CONFIGURACIÓN ==================
# 👉 API Key de Gemini (puede usar la nuestra o crear la suya en https://aistudio.google.com/)"
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# 👉 Configura Neo4j con tus propios datos (SOLO si falla la configuración por defecto del Colab)
NEO4J_URI = userdata.get('urlneo')
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = userdata.get('neo4j')

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# ================== SUBIR DOCUMENTO ==================
print("Sube un archivo de texto con información (ej: innovadores.txt)")
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
with open(file_name, "r", encoding="utf-8") as f:
    text = f.read()

# ================== EXTRAER TRIPLES CON GEMINI ==================
model = genai.GenerativeModel("models/gemini-1.5-flash-latest")

prompt = f"""
Extrae relaciones del siguiente texto en formato de triples:
(SUJETO, RELACIÓN, OBJETO).
Texto:
{text}
"""

response = model.generate_content(prompt)
print("🔹 Triples extraídos por Gemini:")
print(response.text)

# ================== GUARDAR TRIPLES EN NEO4J ==================
def insert_triple(tx, s, r, o):
    query = """
    MERGE (a:Entidad {name: $s})
    MERGE (b:Entidad {name: $o})
    MERGE (a)-[rel:RELACION {tipo: $r}]->(b)
    """
    tx.run(query, s=s, r=r, o=o)

triples = []
for line in response.text.split("\n"):
    if "(" in line and ")" in line:
        line = line.strip("()")
        parts = [p.strip() for p in line.split(",")]
        if len(parts) == 3:
            triples.append(parts)

with driver.session() as session:
    for s, r, o in triples:
        session.write_transaction(insert_triple, s, r, o)

print(f"✅ Se insertaron {len(triples)} triples en Neo4j")

# ================== CONSULTA AL GRAFO ==================
def query_graph(query):
    with driver.session() as session:
        result = session.run(query)
        return [dict(r) for r in result]

# AQUI PUEDEN CAMBIAR LAS QUERY POR LOS EJEMPLOS DE ARRIBA ============================== ⏰⏰⏰ =============================
cypher_query = """
MATCH (p:Entidad)-[r:RELACION]->(c:Entidad)
RETURN p.name AS persona, r.tipo AS relacion, c.name AS compania
"""

results = query_graph(cypher_query)

print("\n🔹 Resultados de la consulta Cypher:")
for r in results:
    print(f"{r['persona']} {r['relacion']} {r['compania']}")

# ================== GEMINI PARA RESPUESTA FINAL ==================
context = "\n".join([f"{r['persona']} {r['relacion']} {r['compania']}" for r in results])

# AQUI PUEDEN CAMBIAR LAS PREGUNTAS POR LOS EJEMPLOS DE ARRIBA ============================== ⏰⏰⏰ =============================

final_prompt = f"""
Basándote en el siguiente contexto de un grafo de conocimiento, responde la pregunta y proporciona contexto:

Contexto:
{context}

Pregunta:  ¿Cuáles son las empresas fundadas por Elon Musk y por Steve Jobs, a qué se dedica cada una y por qué es importante lo que hacen?
"""

final_prompt_no_context = f"""
Basándote en el siguiente contexto de un grafo de conocimiento, responde la pregunta y proporciona contexto:

Pregunta:  ¿Cuáles son las empresas fundadas por Elon Musk y por Steve Jobs, a qué se dedica cada una y por qué es importante lo que hacen?
"""

response_final = model.generate_content(final_prompt)

response_no_context = model.generate_content(final_prompt_no_context)

respuesta_chat = """Elon Musk fundó Tesla y SpaceX, además de participar en la creación de PayPal.

* **Tesla:** Es una compañía dedicada a la fabricación de automóviles eléctricos y soluciones de energía renovable. Su importancia radica en su contribución a la lucha contra el cambio climático mediante la promoción de vehículos eléctricos y energías limpias, además de impulsar la innovación en el sector automotriz.

* **SpaceX:** Es una empresa de exploración espacial que desarrolla y lanza cohetes y satélites. Su importancia reside en su objetivo de reducir los costos del acceso al espacio, facilitar la colonización de Marte y promover la innovación tecnológica en el sector aeroespacial. Contribuye a la exploración espacial y al desarrollo de nuevas tecnologías.

* **PayPal:** Es un sistema de pagos en línea que transformó la manera en que se realizan transacciones digitales. Su importancia se encuentra en haber ofrecido una forma segura, rápida y global de realizar pagos por internet, siendo pionera en la economía digital.

Steve Jobs fue cofundador de Apple.

* **Apple:** Es una empresa reconocida por sus productos tecnológicos innovadores como el iPhone, iPad y MacBook. Su importancia se debe a su influencia en la industria de los dispositivos inteligentes, al establecer estándares de diseño y usabilidad, y revolucionar la manera en que las personas interactúan con la tecnología. Ha sido clave en el desarrollo de esta industria, popularizando conceptos como la interfaz gráfica intuitiva y la integración de software y hardware."""

print("\n🔹 Respuesta generada por Gemini:")
print(response_final.text)


print("\n🔹 Respuesta generada por Gemini sin contexto:")
print(response_no_context.text)


print("\n🔹 Respuesta generada por Chatgpt:")
print(response_chat)


Sube un archivo de texto con información (ej: innovadores.txt)


Saving innovadores.txt to innovadores (8).txt
🔹 Triples extraídos por Gemini:
Aquí tienes las relaciones extraídas del texto en formato de triples (SUJETO, RELACIÓN, OBJETO):

* (Elon Musk, fundó, Tesla)
* (Tesla, tipo_de_compañía, fabricante de automóviles eléctricos)
* (Tesla, se_dedica_a, soluciones de energía renovable)
* (Elon Musk, fundó, SpaceX)
* (SpaceX, tipo_de_empresa, empresa de exploración espacial)
* (SpaceX, desarrolla, cohetes)
* (SpaceX, desarrolla, satélites)
* (Elon Musk, participó_en_la_creación_de, PayPal)
* (PayPal, tipo_de_sistema, sistema de pagos en línea)
* (PayPal, revolucionó, transacciones digitales)
* (Steve Jobs, fue_cofundador_de, Apple)
* (Apple, reconocida_por, productos tecnológicos innovadores)
* (Apple, desarrolló, iPhone)
* (Apple, desarrolló, iPad)
* (Apple, desarrolló, MacBook)
* (Apple, fue_clave_en, desarrollo de la industria de los dispositivos inteligentes)
* (Jeff Bezos, fundó, Amazon)
* (Amazon, inicio_como, librería en línea)
* (Amazon, ev

/tmp/ipython-input-4281262237.py:61: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_triple, s, r, o)


✅ Se insertaron 35 triples en Neo4j

🔹 Resultados de la consulta Cypher:
* **(Elon Musk fundó Tesla)**
* **(Tesla tipo_de_compañía fabricante de automóviles eléctricos y soluciones de energía renovable)**
* **(Elon Musk fundó SpaceX)**
* **(SpaceX tipo_de_empresa empresa de exploración espacial)**
* **(SpaceX desarrolla cohetes y satélites)**
* **(Elon Musk participó_en_la_creación PayPal)**
* **(PayPal tipo_de_servicio sistema de pagos en línea)**
* **(Steve Jobs cofundó Apple)**
* **(Apple tipo_de_empresa empresa reconocida por sus productos tecnológicos innovadores)**
* **(Apple desarrolló iPhone)**
* **(Apple desarrolló iPad)**
* **(Apple desarrolló MacBook)**
* **(Apple clave_en_el_desarrollo industria de los dispositivos inteligentes)**
* **(Jeff Bezos fundó Amazon)**
* **(Amazon inicio_como librería en línea)**
* **(Amazon evolucionó_a plataforma de comercio electrónico y servicios en la nube)**
* **(Amazon ofrece Amazon Web Services (AWS))**
* **(Mark Zuckerberg creó Facebook)*

### Métricas de evaluación

In [59]:
# ========= 8. COMPARACIÓN E INTERPRETACIÓN DE RESULTADOS =========

print("\n=== Comparación e Interpretación de las Métricas ===")

print("\nComparando las respuestas generadas con la referencia (ChatGPT):")

bue_score_context, rouge_score_context = evaluate_response(response, respuesta_chat)
bue_score_no_context, rouge_score_no_context = evaluate_response_no_context(response_no_context, respuesta_chat)

# Compare BLEU scores
print("\nResultados BLEU:")
print(f"- Gemini con contexto (BLEU): {bleu_score_context['bleu']:.4f}")
print(f"- Gemini sin contexto (BLEU): {bleu_score_no_context['bleu']:.4f}")

if bleu_score_context['bleu'] > bleu_score_no_context['bleu']:
    print("  -> Según el BLEU score general, la respuesta de Gemini con contexto es más similar a la referencia.")
elif bleu_score_context['bleu'] < bleu_score_no_context['bleu']:
     print("  -> Según el BLEU score general, la respuesta de Gemini sin contexto es más similar a la referencia.")
else:
    print("  -> Los BLEU scores generales son similares.")

print("\nResultados ROUGE:")
print(f"- Gemini con contexto (ROUGE-1): {rouge_score_context['rouge1']:.4f}")
print(f"- Gemini sin contexto (ROUGE-1): {rouge_score_no_context['rouge1']:.4f}")
print(f"- Gemini con contexto (ROUGE-L): {rouge_score_context['rougeL']:.4f}")
print(f"- Gemini sin contexto (ROUGE-L): {rouge_score_no_context['rougeL']:.4f}")


if rouge_score_context['rouge1'] > rouge_score_no_context['rouge1']:
    print("  -> Según ROUGE-1 (superposición de palabras individuales), la respuesta de Gemini con contexto es más similar a la referencia.")
elif rouge_score_context['rouge1'] < rouge_score_no_context['rouge1']:
    print("  -> Según ROUGE-1, la respuesta de Gemini sin contexto es más similar a la referencia.")
else:
    print("  -> Los ROUGE-1 scores son similares.")

if rouge_score_context['rougeL'] > rouge_score_no_context['rougeL']:
    print("  -> Según ROUGE-L (subsecuencia más larga, sensible al orden), la respuesta de Gemini con contexto es más similar a la referencia.")
elif rouge_score_context['rougeL'] < rouge_score_no_context['rougeL']:
    print("  -> Según ROUGE-L, la respuesta de Gemini sin contexto es más similar a la referencia.")
else:
     print("  -> Los ROUGE-L scores son similares.")


=== Comparación e Interpretación de las Métricas ===

Comparando las respuestas generadas con la referencia (ChatGPT):

🔹 BLEU score (Gemini with context): 0.0000
🔹 BLEU precision scores (Gemini with context):
- Precision@1: 0.0960
- Precision@2: 0.0067
- Precision@3: 0.0000
- Precision@4: 0.0000

🔹 ROUGE score (Gemini with context):
- rouge1: 0.2222
- rouge2: 0.0260
- rougeL: 0.1292
- rougeLsum: 0.1499

🔹 BLEU score (Gemini without context): 0.0000

🔹 ROUGE score (Gemini without context):
- rouge1: 0.1965
- rouge2: 0.0117
- rougeL: 0.1272
- rougeLsum: 0.1272

Resultados BLEU:
- Gemini con contexto (BLEU): 0.0904
- Gemini sin contexto (BLEU): 0.0896
  -> Según el BLEU score general, la respuesta de Gemini con contexto es más similar a la referencia.

Resultados ROUGE:
- Gemini con contexto (ROUGE-1): 0.2222
- Gemini sin contexto (ROUGE-1): 0.1965
- Gemini con contexto (ROUGE-L): 0.1292
- Gemini sin contexto (ROUGE-L): 0.1272
  -> Según ROUGE-1 (superposición de palabras individuales),

# 3. **FusionRAG (BM25 + Embeddings)**

En este ejemplo se combina lo mejor de dos enfoques de recuperación de información:

- BM25 (keyword-based): Recupera pasajes basándose en la coincidencia de palabras clave.

- Vector Search (embeddings con ChromaDB): Recupera fragmentos usando similitud semántica.

Ambos resultados se fusionan para obtener un contexto más robusto y completo, que luego se pasa al modelo Gemini para generar una respuesta.

👉 Deben subir un archivo de texto (ej: cuantica.txt, historia_colombia.txt, innovadores.txt o historia_internet.txt) y luego probar con diferentes preguntas modificando el campo query.

## **Preparación del entorno:**

Si ya configuraste los entornos de los ejemplos anteriores, no necesitas hacer nada adicional para este bloque. 🚀

In [64]:
# ================= DEMO FUSION RAG ==================
!pip install rank_bm25 chromadb sentence-transformers google-generativeai

from rank_bm25 import BM25Okapi
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import chromadb
import os
from google.colab import files


# ================= SUBIR DOCUMENTO ==================
print("📂 Sube un archivo de texto con información (ej: cuantica.txt, historia_colombia.txt, etc.)")
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
with open(file_name, "r", encoding="utf-8") as f:
    text = f.read()

# ================= CHUNKING ==================
def chunk_text(text, chunk_size=80, overlap=20):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size - overlap)]

docs = chunk_text(text)

# ========= Vector Store (embeddings con ChromaDB)
embedder = SentenceTransformer("all-MiniLM-L6-v2")
client = chromadb.Client()
collection = client.get_or_create_collection("fusion_chunks")
embeddings = embedder.encode(docs).tolist()
for i, d in enumerate(docs):
    collection.add(documents=[d], embeddings=[embeddings[i]], ids=[str(i)])

# ========= BM25 retriever
tokenized_corpus = [d.split(" ") for d in docs]
bm25 = BM25Okapi(tokenized_corpus)

# ========= Fusion Retrieval
query = "¿Qué científico propuso un modelo atómico en 1913?"
q_embed = embedder.encode([query]).tolist()
results_vector = collection.query(query_embeddings=q_embed, n_results=3)
results_bm25 = bm25.get_top_n(query.split(" "), docs, n=3)

# Fusión (simple: concatenación + eliminación de duplicados)
fusion_results = list(set(results_vector['documents'][0] + results_bm25))
retrieved_context = " ".join(fusion_results)

# ================= GEMINI PARA RESPUESTA FINAL ==================
response = model.generate_content(
    f"Basándote en el siguiente contexto (resultado de fusión de múltiples recuperadores), responde la pregunta y añade explicación:\n\n{retrieved_context}\n\nPregunta: {query}"
)


response_no_context = model.generate_content(
    f"Basándote en el siguiente contexto (resultado de fusión de múltiples recuperadores), responde la pregunta y añade explicación:\n\nPregunta: {query}"
)

response_chat = """El científico que propuso un modelo atómico en 1913 fue Niels Bohr

Bohr planteó que los electrones no podían ocupar cualquier órbita alrededor del núcleo, sino que se encontraban en niveles de energía cuantizados. Esto significaba que los electrones solo podían moverse entre niveles discretos, emitiendo o absorbiendo energía en forma de cuantos (fotones). Su modelo fue una mejora respecto al de Rutherford, ya que explicaba fenómenos como el espectro del hidrógeno, donde las líneas de emisión correspondían a transiciones electrónicas entre estos niveles.

Este aporte fue crucial porque introdujo el concepto de cuantización al modelo atómico, conectando las ideas de Planck y Einstein con la estructura de la materia, y sentando las bases de la mecánica cuántica moderna."""

print("\n🔹 Respuesta generada con FusionRAG:")
print(response.text)

print("\n🔹 Respuesta generada por Gemini sin contexto:")
print(response_no_context.text)


print("\n🔹 Respuesta generada por Chatgpt:")
print(response_chat)



📂 Sube un archivo de texto con información (ej: cuantica.txt, historia_colombia.txt, etc.)


Saving cuantica.txt to cuantica (3).txt

🔹 Respuesta generada con FusionRAG:
Niels Bohr propuso un modelo atómico en 1913.  El texto explícitamente menciona que en ese año, Bohr propuso su modelo atómico donde los electrones orbitaban en niveles de energía cuantizados alrededor del núcleo.  La información sobre la independencia de Colombia es irrelevante para responder a esta pregunta específica.


🔹 Respuesta generada por Gemini sin contexto:
El científico que propuso un modelo atómico en 1913 fue **Niels Bohr**.

La explicación reside en que el modelo atómico de Bohr, publicado en 1913,  fue una mejora significativa al modelo de Rutherford.  Si bien el modelo de Rutherford describió correctamente la existencia de un núcleo atómico positivo con electrones orbitándolo, no explicaba la estabilidad del átomo.  El modelo de Bohr incorporó la teoría cuántica de Planck, postulando que los electrones orbitan el núcleo en órbitas específicas con niveles de energía definidos, y que los electro

### Métricas de evaluación

In [65]:
# ========= 8. COMPARACIÓN E INTERPRETACIÓN DE RESULTADOS =========

print("\n=== Comparación e Interpretación de las Métricas ===")

print("\nComparando las respuestas generadas con la referencia (ChatGPT):")

bue_score_context, rouge_score_context = evaluate_response(response, response_chat)
bue_score_no_context, rouge_score_no_context = evaluate_response_no_context(response_no_context, response_chat)

# Compare BLEU scores
print("\nResultados BLEU:")
print(f"- Gemini con contexto (BLEU): {bleu_score_context['bleu']:.4f}")
print(f"- Gemini sin contexto (BLEU): {bleu_score_no_context['bleu']:.4f}")

if bleu_score_context['bleu'] > bleu_score_no_context['bleu']:
    print("  -> Según el BLEU score general, la respuesta de Gemini con contexto es más similar a la referencia.")
elif bleu_score_context['bleu'] < bleu_score_no_context['bleu']:
     print("  -> Según el BLEU score general, la respuesta de Gemini sin contexto es más similar a la referencia.")
else:
    print("  -> Los BLEU scores generales son similares.")

print("\nResultados ROUGE:")
print(f"- Gemini con contexto (ROUGE-1): {rouge_score_context['rouge1']:.4f}")
print(f"- Gemini sin contexto (ROUGE-1): {rouge_score_no_context['rouge1']:.4f}")
print(f"- Gemini con contexto (ROUGE-L): {rouge_score_context['rougeL']:.4f}")
print(f"- Gemini sin contexto (ROUGE-L): {rouge_score_no_context['rougeL']:.4f}")


if rouge_score_context['rouge1'] > rouge_score_no_context['rouge1']:
    print("  -> Según ROUGE-1 (superposición de palabras individuales), la respuesta de Gemini con contexto es más similar a la referencia.")
elif rouge_score_context['rouge1'] < rouge_score_no_context['rouge1']:
    print("  -> Según ROUGE-1, la respuesta de Gemini sin contexto es más similar a la referencia.")
else:
    print("  -> Los ROUGE-1 scores son similares.")

if rouge_score_context['rougeL'] > rouge_score_no_context['rougeL']:
    print("  -> Según ROUGE-L (subsecuencia más larga, sensible al orden), la respuesta de Gemini con contexto es más similar a la referencia.")
elif rouge_score_context['rougeL'] < rouge_score_no_context['rougeL']:
    print("  -> Según ROUGE-L, la respuesta de Gemini sin contexto es más similar a la referencia.")
else:
     print("  -> Los ROUGE-L scores son similares.")


=== Comparación e Interpretación de las Métricas ===

Comparando las respuestas generadas con la referencia (ChatGPT):

🔹 BLEU score (Gemini with context): 0.0502
🔹 BLEU precision scores (Gemini with context):
- Precision@1: 0.6154
- Precision@2: 0.2745
- Precision@3: 0.1600
- Precision@4: 0.1020

🔹 ROUGE score (Gemini with context):
- rouge1: 0.3854
- rouge2: 0.2000
- rougeL: 0.2188
- rougeLsum: 0.2812

🔹 BLEU score (Gemini without context): 0.1580

🔹 ROUGE score (Gemini without context):
- rouge1: 0.5051
- rouge2: 0.2405
- rougeL: 0.3276
- rougeLsum: 0.3823

Resultados BLEU:
- Gemini con contexto (BLEU): 0.0904
- Gemini sin contexto (BLEU): 0.0896
  -> Según el BLEU score general, la respuesta de Gemini con contexto es más similar a la referencia.

Resultados ROUGE:
- Gemini con contexto (ROUGE-1): 0.3854
- Gemini sin contexto (ROUGE-1): 0.5051
- Gemini con contexto (ROUGE-L): 0.2188
- Gemini sin contexto (ROUGE-L): 0.3276
  -> Según ROUGE-1, la respuesta de Gemini sin contexto es m